As an additional exploration with the species identified as having GAP habitat maps available, this notebook explores adding a view of GAP habitat metrics to the cache of information. This may be useful in determining where else species may have range within states. The process uses the 2018 US State boundaries from Census, the GAP range bounding box (generated previously in the basic caching of GAP species data from the GAP range services), and a set of pre-calculated GAP metrics that include acres within specific protection status designations in each state. It first finds a basic intersection of the range bounding box with a set of states, retrieves the total state metrics for all GAP species, and then filters to an individual species (no current query criteria for this at the API). This information is then cached for further reference.

In [1]:
import geopandas as gpd
import json
import requests
from IPython.display import display
import bispy
from joblib import Parallel, delayed

gap = bispy.gap.Gap()
bis_utils = bispy.bis.Utils()

In [2]:
us_states = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2018/STATE/tl_2018_us_state.zip")

In [3]:
# Open the file back up and verify
with open("cache/gap.json", "r") as f:
    gap_cache = json.loads(f.read())


In [ ]:
gap_metrics_cache = list()
for gap_species in gap_cache:
    gap_metrics_cache.append(gap.gap_metrics_species(us_states, gap_species["GAP Species"]["GAP_SpeciesCode"], gap_species["GAP Species"]["Range Bounding Box"]))


In [4]:
%%time
# Use joblib to run multiple requests for SGCN records in parallel via scientific names
gap_metrics = Parallel(n_jobs=8)(delayed(gap.gap_metrics_species)(us_states, gap_species["GAP Species"]["GAP_SpeciesCode"], gap_species["GAP Species"]["Range Bounding Box"]) for gap_species in gap_cache)


CPU times: user 4.79 s, sys: 2.68 s, total: 7.46 s
Wall time: 1min 47s


In [5]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/gap_metrics.json", gap_metrics))

{'Doc Cache File': 'cache/gap_metrics.json',
 'Document Number 8': {'GAP_SpeciesCode': 'aCBSAx',
  'State Metrics': [{'gapstat123ac': 143.2222788276,
    'gapstat123group': '1-10',
    'gapstat123perc': 1.81474906303717,
    'gapstat12ac': 98.0761257189,
    'gapstat12group': '1-10',
    'gapstat12perc': 1.24270859751458,
    'gapstat1ac': 0.0,
    'gapstat1perc': 0.0,
    'gapstat2ac': 98.0761257189,
    'gapstat2perc': 1.24270859751458,
    'gapstat3ac': 45.1461531087,
    'gapstat3perc': 0.572040465522586,
    'gapstat4ac': 7748.9035111647,
    'gapstat4perc': 98.1852509369628,
    'id': 20112,
    'spp_comname': 'Comal Blind Salamander',
    'spp_sciname': 'Eurycea tridentifera',
    'sppcode': 'aCBSAx',
    'state_fipscode': '48',
    'state_name': 'Texas',
    'taxa': 'A',
    'totalac': 7892.1257899923}]},
 'Number of Documents in Cache': 63}